In [1]:
#################################################################
# This script is for generating training dataset from 
# original .csv file by standardizing product_type and 
# labelling the first class label by algorithms.
#
# Please make sure the input file path and type is correct
# Also, make sure product_to_all.json and main_to_num.json
# in the same directory(or same path.) Since these two files
# are the standardize information we use and the product label 
# map to number we predefined.
# 
# Author: Luis Lin
# Date: June 27, 2022
#################################################################

In [1]:
import json
import spacy 
from difflib import SequenceMatcher
from typing import List
import pandas as pd
from string_grouper import match_strings, match_most_similar
import numpy as np
import text_cleaner as tc
import re
#the main_categories with all sub_categories dict
products_to_all = {}
#the main_categories relate to numbers dict
main_categories_map_to_num = {}
#all categories of product including the name of main_cat
all_cat = set()
#each specific item maps to the main_categories_number
specific_products_map_to_num = {}
#generate label_2nd class from specific labels
label3_to_label2_map = {}


'''
Input: two separate strings
Function:
    We provide another statistic model (ML) from spacy.similarity to compare two strings similarity firstly.
    if spacy similarities doesn't not exit like 0.0, then we compare two strings similarity by "gestalt pattern matching" 
    not_Statistic model(ML)  It is a character-based matcher. 
Output: Similarity [0, 1] between those two strings
'''
def similar(a: str, b: str) -> float:
    nlp = spacy.load("en_core_web_md")
    doc1 = nlp(a)
    doc2 = nlp(b)
    statistical_method_score = doc1.similarity(doc2)
    non_statistical_method_score = SequenceMatcher(None, a, b).ratio()
    if statistical_method_score < 0.1:
        return non_statistical_method_score
    return statistical_method_score

'''
Input: string
Function: remove any plural format and wired suffix, for example, dresses -> ['dress']
Output: list 
'''
def lemma_string(original_string:str) -> List:
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(original_string.lower())
    temp = []
    for token in doc:
        if token.text == '&' or token.text == 'and' or token.text == '/' or token.text == ',':
            continue
        if token.text == "glasses" or token.text == 'booty' or token.text == "sunglasses" or token.text == "earrings": #some of them should be represented in plural format
            temp.append(token.text)
        else:
            temp.append(token.lemma_)
    return temp
'''
input: original string
function: remove pre_suffix of the original 'product_type' to match more accurate items
output: modified string
'''
def modify_product_type(original:str):
    if len(original) <= 1:
        return original
    removable_words = ["clothing", "sale", "man", "men","mens", "men's", "woman", "women", "womens", "women's",  "unisex",\
        "girl", "girls", "girl's", "lady", "ladies", "snow","ladies'", "active","boy", "boys", "boy's", "graphic", "premium", \
            "cozy", "designer","comfort", "athletic","casual", 'youth', 'adult','formal']
    original = original.replace("+", " ").replace("-", " ").replace("&", " ").replace("/", " ").replace(",", " ")
    original = re.compile(r"\s+").sub(" ", original).strip()
    ori_list = original.split(" ")
    
    acc = ['acc', 'accessory', 'acessories','accesssories','accessories', "jewelry","polarized", "non polar"]
    shoes = ['footwear', "shoes"]
    homeware = ["home", "homeware"]
    tops = ["top", "tops"]
    bottoms = ["bottoms", "bottom","sweetlegs"]
    main_cat = ''
    sub_cat = []
    
    res = []
    
    #Get rid of "bikini top and bikini bottoms" from tops and bottoms, 
    # because they belong to swimwear.
    other_clothing_sensative_words = ["bikini", "swim", "swimwear", "tankini"]
    is_other_clothing = False
    sub = []
    for e in ori_list:
        e = e.strip().lower()
        if e in removable_words:
            continue
        elif not res.__contains__(e):
            res.append(e)
        if e in acc:
            main_cat = "accessories"
        elif e in shoes:
            main_cat = "shoes"
        elif e in tops:
            main_cat = "tops"
        elif e in bottoms:
            main_cat = "bottoms"
        elif e in homeware:
            main_cat = "homeware"
        elif e == "beauty":
            main_cat = "beauty"
        elif e in other_clothing_sensative_words :
            is_other_clothing = True
            if e not in sub_cat:
                sub_cat.append(e)
        else:
            #the rest of string can be added to sub_category
            if e not in sub_cat:
                sub_cat.append(e)
        if is_other_clothing:
            if main_cat != '' and main_cat != 'other_clothing':
                sub_cat.append(main_cat)
            main_cat = "other_clothing"


    #if main_cat doesn't exist but sub_cat exists, we can try to map directly by specific_product_map
    sub_cat = " ".join(sub_cat)
    
    return " ".join(res), main_cat, sub_cat


'''
Input: None
Function: Read the .json file to initilize the dictionaries and sets
Output: None
'''
def initiailize_containers() -> None:
    with open("json_files/product_to_all.json") as f1:
        global products_to_all 
        products_to_all = json.load(f1)
    with open("json_files/main_categories_to_num.json") as f2: 
        global main_categories_map_to_num
        main_categories_map_to_num = json.load(f2)
    with open("json_files/specific_product_map_to_num.json") as f3:
        global specific_products_map_to_num
        specific_products_map_to_num = json.load(f3)
    for key in specific_products_map_to_num.keys():
        all_cat.add(key)
    with open("json_files/label3_to_label2.json") as f4:
        global label3_to_label2_map
        label3_to_label2_map = json.load(f4)
        
    
'''
Input: dataframe
Function: to calculate some statistics data after pre-processing.
'''
def summary_of_the_new_df(df:pd.DataFrame)->None:
    unknown = 0
    totalnum = df.shape[0]
    shoes = 0
    other_clothing = 0
    tops = 0
    bottoms = 0
    beauty = 0
    home = 0
    acc = 0
    other = 0
    label2_count = 0
    for row in range(df.shape[0]):
        n = df.loc[row, "label_1st"]
        if n == 0:
            unknown += 1
        elif n == 1:
            shoes += 1
        elif n == 2:
            tops += 1
        elif n == 3:
            bottoms += 1
        elif n == 4:
            other_clothing += 1
        elif n == 5:
            beauty += 1
        elif n == 6:
            acc += 1
        elif n == 7:
            home += 1
        elif n == 8:
            other += 1
        m = df.loc[row, "label_2nd"]
        if m != "":
            label2_count += 1
        
    print("unknown", unknown, "\t\tratio of all", unknown/totalnum)
    total = totalnum - unknown
    print("lable_2 ", label2_count, "\t\tratio", "{:10.2f}".format(label2_count/total) )
    print("\nshoes", shoes,"\t\tratio", "{:10.2f}".format(shoes/total))
    print("tops", tops, "\t\tratio", "{:10.2f}".format(tops/total))
    print("bottoms", bottoms, "\t\tratio", "{:10.2f}".format(bottoms/total))
    print("other_clothing", other_clothing,"\tratio", "{:10.2f}".format(other_clothing/total))
    print("beauty", beauty, "\t\tratio", "{:10.2f}".format(beauty/total))
    print("accessories", acc, "\tratio", "{:10.2f}".format(acc/total))
    print("homeware", home, "\t\tratio", "{:10.2f}".format(home/total))
    print("other", other, "\t\tratio", "{:10.2f}".format(other/total))
#iniyilize all maps and sets.
initiailize_containers()

Init Plugin
Init Graph Optimizer
Init Kernel


In [3]:
original_file_path = "/Users/luis/Downloads/products-June-28th.csv"
assert original_file_path != None
columns = ["id", "title", "tags", "images", "gender","product_type",  "colors", "buckets", "url", "body_html"]
df = pd.DataFrame()
try:
    df = pd.read_csv(original_file_path, usecols=columns).reset_index()
except:
    print("The path seems incorrect")
#insert new columns to this df which are useful and processed information 

df.insert(df.columns.get_loc("gender"), "product_type(modified)", "", allow_duplicates=True)
df.insert(df.columns.get_loc("gender"), "main_category", "", allow_duplicates=True)
df.insert(df.columns.get_loc("gender"), "sub_category", "", allow_duplicates=True)
df.insert(df.columns.get_loc("gender"), "match_most_similar_>80%_string", "", allow_duplicates=True)
df.insert(df.columns.get_loc("gender"), "match_most_similar_>60%_string", "", allow_duplicates=True)
df.insert(df.columns.get_loc("gender"), "label_1st", 0, allow_duplicates=True)
df.insert(df.columns.get_loc("gender"), "label_2nd", "", allow_duplicates=True)
df.insert(df.columns.get_loc("gender"), "label_3rd", 0, allow_duplicates=True)
df.insert(df.columns.get_loc("buckets"), "buckets_num", 0, allow_duplicates=True)
df.insert(df.columns.get_loc("url"), "color_num", 0, allow_duplicates=True)
df = df[["index","id", "title", "tags", "images", "gender","product_type", "product_type(modified)", "main_category", "sub_category", "match_most_similar_>80%_string", "match_most_similar_>60%_string", "label_1st", "label_2nd","label_3rd", "buckets_num", "buckets", "color_num", "colors", "url", "body_html"]]
df.insert(df.columns.get_loc("body_html"), "raw_text", "", allow_duplicates=True)
for i in range(df.shape[0]):
    
    ori_word = df.loc[i, 'product_type']
    title = df.loc[i, 'title']
    tags = df.loc[i, 'tags']
    buckets = df.loc[i, 'buckets']
    body_html = df.loc[i, 'body_html']
    
    if not isinstance(ori_word, str):
        df.loc[i,['product_type']] = "unknown"
        df.loc[i, ['product_type(modified)']] = "unknown"
        continue
    #get the modify_product_type (remove)
    try:
        product_type_new, main_cat, sub_cat= modify_product_type(ori_word)
        df.loc[i, ['product_type(modified)']] = product_type_new
        df.loc[i, ["main_category"]] = main_cat
        df.loc[i, ["sub_category"]] = sub_cat
        
        if (main_cat != ""):
            #label the 1st, label if we already know exact main_catgories. 
            df.loc[i,["label_1st"]] = main_categories_map_to_num.get(main_cat)
    except:
        print(i, ori_word)
    try:
        #clean body_html as well. 
        df.loc[i, 'raw_text'] = tc.cleanHtml(body_html)
    except:
        print("Something wrong with clean html: ", i)
del(df["body_html"])

In [4]:
#build data series from all categories.
pre_defiened_labels = pd.Series(list(all_cat), name="pre_defined_label")
#get 80% most_similar_mathes(dataframe) by using package string-matcher
most_similar_matches = match_most_similar( pre_defiened_labels, df["sub_category"],\
    min_similarity = 0.80, ignore_index=False, replace_na=False)
most_similar_matches = pd.concat([df['index'],df["sub_category"], most_similar_matches], axis=1)
#get 60% most_similar_matches(dataframe) 
less_similar_matches = match_strings(pre_defiened_labels, df["sub_category"],\
    min_similarity = 0.65, ignore_index = False, replace_na = False)
#fill up 80% most_similar_80% column
empty_cells = 0
for row in range(most_similar_matches.shape[0]):
    index = most_similar_matches.loc[row, "index"]
    most_similar_pre_defined_label = most_similar_matches.loc[row, 'most_similar_pre_defined_label']
    most_similar_index = most_similar_matches.loc[row, ['most_similar_index']].item()
    
    if not np.isnan(most_similar_index):
        df.loc[df['index'] == index, ['match_most_similar_>80%_string']] = most_similar_pre_defined_label
    else:
        empty_cells += 1
#fill up 60% less_similar_60% column
index_similarity_map = {}
index_to_pre_map = {}
for row in range(less_similar_matches.shape[0]):
    current_index = less_similar_matches.loc[row, 'right_index']
    label = less_similar_matches.loc[row, 'left_pre_defined_label']
    similarity = less_similar_matches.loc[row, 'similarity']
    
    if index_similarity_map.get(current_index) == None:
        index_similarity_map.update({current_index:similarity})
        index_to_pre_map.update({current_index:label})
    elif index_similarity_map.get(current_index) >= similarity:
        index_similarity_map.update({current_index:similarity})
        index_to_pre_map.update({current_index:label})
    else: # similarity < then current, ignore
        continue
for key in index_to_pre_map.keys():
    df.loc[df['index'] == key, ['match_most_similar_>60%_string']] = index_to_pre_map.get(key)

In [5]:
#Algorithm 2: if both 60% and 80% don't exist, just ignore, 
#             if the 80% string doesn't exist, compare the 60% string with 80% string similarity by diff() and spacy(), 
#             let spacy.similarity to decide whether above 60% to fill the 80%. if spacy similarity does not exist, fill 
#             fill up 80% by similar() algorithm. 
#     speed : 500 data around 1 mins based on M1 pro.
double_check_pairs_pt_word60_dic = {}
#see the accurancy 

def is_nan_string(string):
    return string == ""
#w = df.loc[66, ["match_most_similar_>80%_string"]].item()
#print(w)
#is_nan_string(w)
for row in range(df.shape[0]):
    word_80 = df.loc[row, "match_most_similar_>80%_string"]
    word_60 = df.loc[row, "match_most_similar_>60%_string"]
    #print(is_nan_string(word_80), "   ",word_60)
    if is_nan_string(word_80) and is_nan_string(word_60):
        #found nothing
        continue    
    elif is_nan_string(word_80) and not is_nan_string(word_60):
        pt_modified = df.loc[row, "sub_category"]
        pair = (pt_modified, word_60)
        #print(pair)
        if pair not in double_check_pairs_pt_word60_dic.keys():
            
            double_check_pairs_pt_word60_dic.update({pair: similar(word_60, pt_modified)})
        #print("product_type(modified) ", pt_modified, " words(60%): ", word_60, " Similarity: ", similar(word_60, pt_modified))
        if similar(word_60, pt_modified) > 0.0:
            df.loc[row, "match_most_similar_>80%_string"] = word_60

/var/folders/r2/xt2ymfnd7nz2khk687zqlrpr0000gn/T/ipykernel_2534/1531767440.py:34: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  statistical_method_score = doc1.similarity(doc2)


In [6]:
double_check_pairs_pt_word60_dic
dict(sorted(double_check_pairs_pt_word60_dic.items(), key=lambda item: item[1]))

{('sandle', 'candle'): 0.10362143556812971,
 ('skateboards', 'skateboard deck'): 0.1400997204021604,
 ('boots', 'boot'): 0.15054492969995684,
 ('tanks', 'tank'): 0.15613156630452035,
 ('jeans', 'jean'): 0.18565857977597314,
 ('vests', 'vest'): 0.18939607925766822,
 ('pants leggings', 'legging'): 0.197313228778295,
 ('watches', 'watch'): 0.21185678398868243,
 ('swim wear tops', 'swimwear'): 0.23622294108489308,
 ('caps', 'cap'): 0.2507941536654137,
 ('jumpsuits rompers', 'jumpsuit romper'): 0.2896266698301745,
 ('rompers jumpsuits', 'jumpsuit romper'): 0.2896266698301745,
 ('chair', 'hair'): 0.3271389297026192,
 ('cocktails', 'cocktail dress'): 0.4654833117992505,
 ('bralette bikini tops', 'bralette'): 0.5745833569330893,
 ('short sleeve button up shirts', 'button-up shirt'): 0.6128854137966951,
 ('long sleeve button up shirts', 'button-up shirt'): 0.6128854137966951,
 ('accessories: cologne', 'accessories'): 0.6322073646834055,
 ('bags packs textile', 'textile'): 0.6335855100915349,
 (

In [2]:
#From previous we finished match similar products. We start to label the "label_1st" column
df = pd.read_csv("(V1.6)all_products_data_set.csv")
for row in range(df.shape[0]):
    key = df.loc[row, 'match_most_similar_>80%_string']
    if key != None and specific_products_map_to_num.get(key) != None:
        df.loc[row, ['label_1st']] = specific_products_map_to_num.get(key)
    if key != None and label3_to_label2_map.get(key) != None:
        df.loc[row, ["label_2nd"]] = label3_to_label2_map.get(key)


In [3]:
df.columns

Index(['index', 'id', 'title', 'tags', 'images', 'gender', 'product_type',
       'product_type(modified)', 'main_category', 'sub_category',
       'match_most_similar_>80%_string', 'match_most_similar_>60%_string',
       'label_1st', 'label_2nd', 'label_3rd', 'buckets_num', 'pattern_info',
       'buckets', 'color_info', 'color_num', 'colors', 'url', 'raw_text'],
      dtype='object')

In [8]:
pd.isna((df.loc[8189]["match_most_similar_>60%_string"]))

True

In [19]:
# Algorithm 3(Optional): If both 80 60 are empty but the sub cat has the entry
# We can map mannually the item to our match_most_similar_>80%_string
for row in range(df.shape[0]):
    #if not pd.isna((df.loc[row]["match_most_similar_>80%_string"])) or not pd.isna((df.loc[row]["match_most_similar_>60%_string"])):
        #continue
    
    sub_category = df.loc[row]["sub_category"]
    if not isinstance(sub_category, str) :
        continue
    elif sub_category.__contains__("bag"):
        df.loc[row, "match_most_similar_>80%_string"] = "bag"
    elif sub_category.__contains__("glove"):
        df.loc[row, "match_most_similar_>80%_string"] = "glove"
        
    
    
#df.to_csv("look.csv", index=False)

In [20]:
for row in range(df.shape[0]):
    key = df.loc[row, 'match_most_similar_>80%_string']
    if key != None and specific_products_map_to_num.get(key) != None:
        df.loc[row, 'label_1st'] = specific_products_map_to_num.get(key)
    if key != None and label3_to_label2_map.get(key) != None:
        df.loc[row, "label_2nd"] = label3_to_label2_map.get(key)
df.to_csv("look.csv", index=False)

In [18]:
summary_of_the_new_df(df)

unknown 7322 		ratio of all 0.2575810877365792
lable_2  28426 		ratio       1.35

shoes 1457 		ratio       0.07
tops 5739 		ratio       0.27
bottoms 3919 		ratio       0.19
other_clothing 1945 	ratio       0.09
beauty 444 		ratio       0.02
accessories 5903 	ratio       0.28
homeware 677 		ratio       0.03
other 1020 		ratio       0.05


In [10]:
df.to_csv("(V1.6)all_products_data_set.csv", index=False)


In [11]:
df.sample(10)

,index,id,title,tags,images,gender,product_type,product_type(modified),main_category,sub_category,...,label_2nd,label_3rd,buckets_num,pattern_info,buckets,color_info,color_num,colors,url,raw_text
22976,22976,7747138912473,Vintage White Crop,{},"[{""src"": ""https://cdn.shopify.com/s/files/1/00...",women,unknown,unknown,NaN,NaN,...,NaN,0,0,NaN,"{Vintage,Trendy}",White (#ffffff),11,{white},https://losttogether.ca/products/vintage-cream...,NaN
7776,7776,6650620346468,"Bamboo Gloves, Brainstorm Bronze","{glove,sale}","[{""src"": ""https://cdn.shopify.com/s/files/1/01...",unset,Accessories - Cover - Gloves,accessories cover gloves,accessories,cover gloves,...,NaN,0,0,NaN,{Casual},Bronze (#cd7f32),6,{bronze},https://blueskyclothingco.com/products/bamboo-...,"Fabric: 95% Bamboo, 5% Lycra"
7043,7043,421432426536,Under Armour Women's Twisted Renegade Cap Steel,"{Accessories,Grey,Hats,Headwear,Training,""Unde...","[{""src"": ""https://cdn.shopify.com/s/files/1/08...",women,Hat,hat,NaN,hat,...,Headwear,0,0,NaN,"{Casual,ActiveWear}",Steel (#737376),5,{steel},https://playstores.ca/products/w-twisted-reneg...,1306297-035 UA Free Fit features a pre-curved ...
23532,23532,7172316201129,70s High Slim Straight,"{22q1,bottoms,denim,levis,womens}","[{""src"": ""https://cdn.shopify.com/s/files/1/00...",unset,Default,default,NaN,default,...,NaN,0,0,marin hits 31l,"{Casual,Trendy}",NaN,0,"{""Marin Hits 31L""}",https://manwomanhome.com/products/70s-high-sli...,This decade is reaching new heights—literally....
16925,16925,6743332716627,LANDYACHTZ Dinghy Classic Maze Cruiser Complete,"{cf-type-cruiser-completes,cf-vendor-landyacht...","[{""src"": ""https://cdn.shopify.com/s/files/1/15...",unset,Cruiser Completes,cruiser completes,NaN,cruiser completes,...,NaN,0,0,NaN,"{Casual,ActiveWear,Streetwear}",Cruiser (#7db138),4,{cruiser},https://freerideboardshop.com/products/landyac...,The Dinghy Classic shape has been a staple of ...
21815,21815,3878815072339,HERSCHEL Retreat Mid Backpack Plum Dot Check,"{ACCBOX20,BF20ACC20,BOXING2020,cf-type-backpac...","[{""src"": ""https://cdn.shopify.com/s/files/1/15...",unset,Backpacks,backpacks,NaN,backpacks,...,Bag,0,0,NaN,"{Casual,ActiveWear,Streetwear}",Check (#5812fd),2,{check},https://freerideboardshop.com/products/hersche...,Signature striped fabric liner Padded and flee...
16528,16528,6755069853779,NIXON DNA Vegan Belt Brown/Gold,"{cf-size-l,cf-size-m,cf-size-s,cf-size-xl,cf-t...","[{""src"": ""https://cdn.shopify.com/s/files/1/15...",men,Men's Belts,belts,NaN,belts,...,Belt,0,0,NaN,"{Casual,ActiveWear,Streetwear}",Gold (#ffd700),12,{gold},https://freerideboardshop.com/products/nixon-d...,A vegan leather strap with logo debossing. Fea...
3660,3660,6963278905511,TANK TOP - WHITE,"{""bruce aop"",floral,gastown,""jack and jones"",s...","[{""src"": ""https://cdn.shopify.com/s/files/1/31...",women,tank top,tank top,tops,tank,...,Tank Tops,0,0,NaN,"{Casual,Bohemian}","White (#ffffff),LRG (#8b31ac)","11,8","{white,LRG}",https://biaboro.ca/products/tank-top-white,White Tank top with floral accent pockets and ...
24674,24674,6632552726720,Pearl Threader Earrings,"{earrings,pearl,threaders}","[{""src"": ""https://cdn.shopify.com/s/files/1/17...",unset,Earrings,earrings,NaN,earrings,...,Jewelry,0,0,NaN,"{Accessories,Bohemian,Trendy}",Pearl (#eae0c8),10,{pearl},https://saltandsage.ca/products/pearl-threader...,“The healing nature of the pearl is highly emo...
2285,2285,4729794297938,Machete Mini Hoops,"{Earrings,Machete}","[{""src"": ""https://cdn.shopify.com/s/files/1/16...",unset,jewelry,jewelry,accessories,NaN,...,NaN,0,0,"blush turtoise,blonde tortoise,ash blonde tort...","{Casual,Trendy}","Cognac (#9a463d),Peach (#fdbb8d),Bright Pink (...","9,6,7,10,10,10,9","{Poppy,""Blonde Tortoise"",Peach,""Blush Turtoise...",https://shopadhoc.com/products/machete-mini-hoops,Ultra-lightweight Mini Hoops are an essential ...
